In [2]:
import pandas as pd
import numpy as np
from psaw import PushshiftAPI

# Getting all submissions

In [18]:
# Initialize API
api = PushshiftAPI()

# Set up generator to make API request.
api_request_generator = api.search_submissions(subreddit='news', score = ">2000")

# Make the request and collect results into a pd.DataFrame
news_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])

# Shape (n_results, n_features)
news_submissions.shape

(14844, 98)

In [19]:
# Available features
news_submissions.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'edited', 'full_link', 'gilded', 'gildings',
       'id', 'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'thumbnail', 'title', 'total_awards_received',
       'treatment_ta

In [17]:
# Transform to datetime
news_submissions['date'] = pd.to_datetime(jokes_submissions['created_utc'], utc = True, unit = 's')
news_submissions[['date','title', 'score', 'num_comments', 'selftext']].sample(20)

,date,title,score,num_comments,selftext
747,2019-07-24 03:28:46+00:00,"My best friend called me and said ""An evil wiz...",12514,117,I drove all the way to his house just to find ...
822,2019-07-10 09:43:25+00:00,I went to the liquor store on my bicycle and b...,6395,93,\n\n...'cause I fell 7 times on the way home...
161,2020-04-15 14:42:22+00:00,Husband doing crossword with his wife,2528,132,"\n\nHusband: Emphatic no, five letters.\n\nWi..."
1312,2019-04-18 20:13:24+00:00,The girl with no arms and legs laying by the pool,9047,238,There’s a girl with no arms and legs laying by...
5600,2016-12-21 09:12:29+00:00,Break ups are the worst in China...,3047,180,You see her face everywhere.
2864,2018-04-05 16:00:16+00:00,I asked my girlfriend to describe me in 5 words.,34538,537,"She said I'm mature, I'm moral, I'm pure, I'm ..."
5254,2017-02-18 22:10:34+00:00,Two blind pilots enter a plane,4247,100,They have sunglasses and white sticks. As the ...
2569,2018-05-19 15:06:42+00:00,Don't ever underestimate a Scottish police off...,5240,144,A London lawyer runs a stop sign and gets pull...
3232,2018-02-12 05:04:36+00:00,Afternoon Sex,36740,663,"The only way to pull off a Sunday afternoon ""q..."
711,2019-07-30 11:44:48+00:00,I once dated a girl who had a twin.,22951,356,People kept asking me how I could tell them ap...


# Getting submission based on search keyword

In [ ]:
# Set up generator to make API request.
api_request_generator = api.search_submissions(q='jobs', score = ">2000")

q_jobs_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])

q_jobs_submissions['date'] = pd.to_datetime(q_jobs_submissionsbmissions['created_utc'], utc=True, unit='s')

